In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import BlackScholes as bs
import ImpliedDrift as iD
import time
import Quintic

from scipy.optimize import least_squares as ls
from matplotlib import cm

In [2]:
dates = np.array(["23_01_23.csv", "24_01_23.csv", "25_01_23.csv", "26_01_23.csv", "27_01_23.csv",
                  "30_01_23.csv", "06_02_23.csv", "13_02_23.csv", "21_02_23.csv"])

spot = np.array(pd.read_csv("spot.csv").Spot).flatten()

moneyness = np.array([80.0,90.0,95.0,97.5,100.0,102.5,105.0,110.0,120.0])

data = pd.read_csv(dates[0])
maturities = np.array(data['Exp Date']).flatten()
IV = np.array(data.drop(columns = 'Exp Date'))/100.

IV = IV[maturities <= 180/365]
maturities = maturities[maturities <= 180/365]

In [3]:
nc = len(dates); start = time.time();
err = np.zeros(nc); param = np.zeros([nc,7]);
inp = np.array([-0.65, 0.05, 0.02, 1, 0.01, 0.02, 0.05]) # Parameter array [rho, H, eps, a0, a1, a3, a5]
bnds = ([-0.999, 0, 0.001, 0, 0, 0, 0],[-1e-9, 0.15, 0.25, np.inf, np.inf, np.inf, np.inf])
N = 12500; n = 1000;

for i in range(nc):
    
    if i:
        data = pd.read_csv(dates[i])
        maturities = np.array(data['Tenor']).flatten()
        IV = np.array(data.drop(columns = ['Date','Tenor']))/100.
        IV = IV[maturities <= 180/365]
        maturities = maturities[maturities <= 180/365]
        
    nr = len(maturities)
    S0 = spot[i]; K = moneyness*S0/100;

    start_time = time.time()
    
    T = maturities[-1]; steps = np.ceil(n*maturities/T).astype(int)
    
    index = i; 
    
    def h(x):
    
        t = time.time()
        np.random.seed(index)
        
        w = np.concatenate((np.zeros([1,N*2]), Quintic.dW(n, N)))
        iv = np.zeros([nr,nc])

        rho, H, eps, a0, a1, a3, a5 = x
        a_k = np.array([a0, a1, a3, a5])

        P = Quintic.global_reduction(rho, H, eps, T, a_k, S0, K, n, N, w, steps, maturities, index)

        for i in range(nr):
            T_aux = maturities[i]
            r = iD.r(T_aux, index); q = iD.q(T_aux, index);
            iv[i,:] = bs.BSImpliedVol(S0, K, T_aux, r, q, P[i], Option_type = 1, toll = 1e-5)

        return iv

    def f(x):
        return h(x).flatten() - IV.flatten()

    result = ls(f, inp, bounds = bnds, max_nfev = 50, ftol = 1e-15, gtol = 1e-15, xtol = 1e-15)
    param[i,:] = result.x
    vol = h(result.x)

    total_time = (time.time() - start_time)/60

    print(f'Execution time date {i+1}: {total_time: .0f} minutes')

    err[i] = np.mean(abs(IV-vol)/IV)*100

    print(f"Relative mean error: {err[i]:.4f}%\n")

print(f"Total execution time: {np.round((time.time()-start)/60): .0f} minutes")

Execution time date 1:  16 minutes
Relative mean error: 2.2917%

Execution time date 2:  15 minutes
Relative mean error: 2.3736%

Execution time date 3:  8 minutes
Relative mean error: 2.6421%

Execution time date 4:  10 minutes
Relative mean error: 3.3515%

Execution time date 5:  12 minutes
Relative mean error: 1.9882%

Execution time date 6:  7 minutes
Relative mean error: 2.0645%

Execution time date 7:  14 minutes
Relative mean error: 2.4261%

Execution time date 8:  12 minutes
Relative mean error: 1.7841%

Execution time date 9:  14 minutes
Relative mean error: 3.7346%

Total execution time:  109 minutes


In [5]:
err

array([2.29172717, 2.37359422, 2.64208479, 3.35147222, 1.9881979 ,
       2.06449225, 2.42605087, 1.78406654, 3.73458234])

In [6]:
param

array([[-8.94220067e-01,  5.51619213e-03,  5.18883238e-02,
         9.53496162e-01,  3.52618783e-01,  2.74474543e-02,
         3.17884966e-02],
       [-8.88246409e-01,  1.09367130e-03,  5.80774709e-02,
         8.63793987e-01,  1.85978741e-01,  1.46250994e-01,
         1.44889851e-02],
       [-8.39822385e-01,  1.66867584e-02,  6.31417516e-03,
         8.12437128e-01,  2.57088357e-01,  2.24362520e-01,
         4.64204120e-02],
       [-8.93449528e-01,  3.64471387e-02,  6.96693577e-03,
         8.26830310e-01,  1.32229508e-01,  1.21523443e-01,
         7.19125016e-02],
       [-8.82400505e-01,  1.93513809e-02,  4.38801500e-02,
         8.10340371e-01,  3.59818499e-01,  1.69325747e-02,
         3.06437429e-02],
       [-8.45566486e-01,  2.52925371e-03,  6.37783816e-02,
         8.26250927e-01,  3.70939103e-01,  4.40625799e-04,
         3.21654476e-02],
       [-9.08754377e-01,  8.86881445e-04,  4.80192181e-02,
         7.81303120e-01,  1.84547821e-01,  8.60680568e-02,
         1.8619975

In [7]:
df = pd.DataFrame(param, columns = ["rho", "H", "eps", "a0", "a1", "a3", "a5"])
df.to_csv("quintic_parameters_bayesian.csv", index = False)

In [8]:
df

,rho,H,eps,a0,a1,a3,a5
0,-0.894220,0.005516,0.051888,0.953496,0.352619,0.027447,0.031788
1,-0.888246,0.001094,0.058077,0.863794,0.185979,0.146251,0.014489
2,-0.839822,0.016687,0.006314,0.812437,0.257088,0.224363,0.046420
3,-0.893450,0.036447,0.006967,0.826830,0.132230,0.121523,0.071913
4,-0.882401,0.019351,0.043880,0.810340,0.359818,0.016933,0.030644
5,-0.845566,0.002529,0.063778,0.826251,0.370939,0.000441,0.032165
6,-0.908754,0.000887,0.048019,0.781303,0.184548,0.086068,0.018620
7,-0.883171,0.003819,0.047489,0.894005,0.341619,0.083400,0.016399
8,-0.942751,0.000981,0.051644,0.898145,0.164039,0.099849,0.025287
